Tutoriel pytesserac : https://www.datacamp.com/fr/tutorial/optical-character-recognition-ocr-in-python-with-pytesseract?dc_referrer=https%3A%2F%2Fwww.google.com%2F


In [1]:
import cv2
import pytesseract

Notre tâche consiste à lire le texte de l'image suivante :

![Image](https://media.datacamp.com/legacy/v1714048579/image_a30b607189.png)

Tout d'abord, nous définissons le chemin de l'image et l'envoyons à la fonction cv2.imread:

In [ ]:
# Read image
easy_text_path = "../data/tutorial/images/easy_text.jpg"
easy_img = cv2.imread(easy_text_path)


Ensuite, nous transmettons l'image chargée à la fonction image_to_string de pytesseract pour extraire le texte :

In [3]:
# Convert to text
text = pytesseract.image_to_string(easy_img)
print(text)


This text is
easy to extract.



C'est aussi simple que cela ! Transformons ce que nous venons de faire en une fonction :

In [4]:
def image_to_text(input_path):
    """
    A function to read text from images.
    """
    img = cv2.imread(input_path)
    text = pytesseract.image_to_string(img)

    return text.strip()


Utilisons la fonction sur une image plus difficile :

![Image](https://media.datacamp.com/legacy/v1714048668/image_9eadc06079.png)

L'image représente un plus grand défi, car elle contient plus de symboles de ponctuation et du texte dans des polices différentes.

In [ ]:
# Define image path
medium_text_path = "../data/tutorial/images/medium_text.jpg"

# Extract text
extracted_text = image_to_text(medium_text_path)
print(extracted_text)

Home >» Tutorials > Data Engineering

Snowflake Tutorial For Beginners:
From Architecture to Running
Databases

Learn the fundamentals of cloud data warehouse management using
Snowflake. Snowflake is a cloud-based platform that offers significant
benefits for companies wanting to extract as much insight from their data as
quickly and efficiently as possible.

Jan 2024 - 12 min read


Notre fonction a fonctionné presque parfaitement. Il a confondu l'un des points et le signe ">", mais le résultat est acceptable par ailleurs.


# <span style='background:blue'>Dessiner des cadres autour du texte</span>

Une opération courante de l'OCR consiste à tracer des boîtes de délimitation autour du texte. Cette opération est prise en charge par PyTesseract.

Tout d'abord, nous transmettons une image chargée à la fonction image_to_data:

In [6]:
from pytesseract import Output

# Extract recognized data from easy text
data = pytesseract.image_to_data(easy_img, output_type=Output.DICT)

La partie Output.DICT garantit que les détails de l'image sont renvoyés sous forme de dictionnaire. Jetons un coup d'œil à l'intérieur :

In [7]:
data

{'level': [1, 2, 3, 4, 5, 2, 3, 4, 5, 5, 5, 4, 5, 5, 5],
 'page_num': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'block_num': [0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 'par_num': [0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'line_num': [0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2],
 'word_num': [0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 3, 0, 1, 2, 3],
 'left': [0, 0, 0, 0, 0, 60, 60, 154, 154, 394, 639, 60, 60, 298, 446],
 'top': [0, 0, 0, 0, 0, 74, 74, 74, 74, 79, 74, 178, 190, 178, 178],
 'width': [859,
  859,
  859,
  859,
  859,
  748,
  748,
  569,
  183,
  185,
  84,
  748,
  184,
  88,
  362],
 'height': [288, 288, 288, 288, 288, 178, 178, 62, 62, 57, 62, 74, 62, 56, 56],
 'conf': [-1, -1, -1, -1, 95, -1, -1, -1, 96, 95, 95, -1, 96, 96, 96],
 'text': ['',
  '',
  '',
  '',
  ' ',
  '',
  '',
  '',
  'This',
  'text',
  'is',
  '',
  'easy',
  'to',
  'extract.']}

Le dictionnaire contient de nombreuses informations sur l'image. Remarquez tout d'abord les touches conf et text. Ils ont tous deux une longueur de 11 :

In [8]:
len(data["text"])

15

Cela signifie que pytesseract a tiré 15 boîtes. Le site conf est synonyme de confiance. S'il est égal à -1, le cadre correspondant est dessiné autour de blocs de texte plutôt que de mots individuels.

Par exemple, si vous regardez les quatre premières valeurs width et height, elles sont grandes par rapport aux autres parce que ces boîtes sont dessinées autour du texte entier au milieu, puis pour chaque ligne de texte et pour l'image globale elle-même.

En outre :

    left est la distance entre le coin supérieur gauche de la boîte englobante et le bord gauche de l'image.
    top est la distance entre le coin supérieur gauche de la boîte englobante et le bord supérieur de l'image.
    width et height sont la largeur et la hauteur de la boîte de délimitation.

À l'aide de ces informations, dessinons les boîtes sur l'image dans OpenCV.

Tout d'abord, nous extrayons à nouveau les données et leur longueur :

In [9]:
from pytesseract import Output

# Extract recognized data
data = pytesseract.image_to_data(easy_img, output_type=Output.DICT)
n_boxes = len(data["text"])

Ensuite, nous créons une boucle pour le nombre de boîtes trouvées :

In [10]:
for i in range(n_boxes):
    if data["conf"][i] == -1:
        continue

À l'intérieur de la boucle, nous créons une condition qui permet d'ignorer l'itération actuelle de la boucle si conf est égal à -1. Le fait de ne pas prendre en compte les boîtes de délimitation plus grandes permet de conserver une image propre.

Ensuite, nous définissons les coordonnées de la boîte actuelle, en particulier les emplacements des coins supérieur gauche et inférieur droit :

In [11]:
for i in range(n_boxes):
    if data["conf"][i] == -1:
        continue
    # Coordinates
    x, y = data["left"][i], data["top"][i]
    w, h = data["width"][i], data["height"][i]

    # Corners
    top_left = (x, y)
    bottom_right = (x + w, y + h)

Après avoir défini certains paramètres de la boîte, tels que sa couleur et son épaisseur en pixels, nous transmettons toutes les informations à la fonction cv2.rectangle:

In [12]:
for i in range(n_boxes):
    if data["conf"][i] == -1:
        continue
    # Coordinates
    x, y = data["left"][i], data["top"][i]
    w, h = data["width"][i], data["height"][i]

    # Corners
    top_left = (x, y)
    bottom_right = (x + w, y + h)

    # Box params
    green = (0, 255, 0)
    thickness = 3  # pixels

    cv2.rectangle(
        img=easy_img, pt1=top_left, pt2=bottom_right, color=green, thickness=thickness
    )

La fonction dessine les cases sur les images originales. Sauvegardons l'image et jetons un coup d'œil :

In [ ]:
# Save the image
output_image_path = "../data/tutorial/images/text_with_boxes.jpg"
cv2.imwrite(output_image_path, easy_img)

True

![Image](https://media.datacamp.com/legacy/v1714049096/image_173eb30226.jpg)

Le résultat est exactement ce que nous voulions !

Maintenant, mettons à nouveau tout ce que nous avons fait dans une fonction :

In [14]:
def draw_bounding_boxes(input_img_path, output_path):
    img = cv2.imread(input_img_path)

    # Extract data
    data = pytesseract.image_to_data(img, output_type=Output.DICT)
    n_boxes = len(data["text"])

    for i in range(n_boxes):
        if data["conf"][i] == -1:
            continue
        # Coordinates
        x, y = data["left"][i], data["top"][i]
        w, h = data["width"][i], data["height"][i]

        # Corners
        top_left = (x, y)
        bottom_right = (x + w, y + h)

        # Box params
        green = (0, 255, 0)
        thickness = 1  # The function-version uses thinner lines

        cv2.rectangle(img, top_left, bottom_right, green, thickness)

    # Save the image with boxes
    cv2.imwrite(output_path, img)

Et utilisez la fonction sur le texte moyennement dur :

In [ ]:
output_path = "../data/tutorial/images/medium_text_with_boxes.png"

draw_bounding_boxes(medium_text_path, output_path)

![Image](https://media.datacamp.com/legacy/v1714049157/image_41aa62ce70.png)

Même pour l'image la plus difficile, le résultat est parfait !

# <span style='background:blue'>Étude de cas : OCR sur un fichier PDF avec Python</span>

Faisons une étude de cas sur un exemple de fichier PDF numérisé. Dans la pratique, il est très probable que vous travailliez avec des PDF numérisés plutôt qu'avec des images, comme celle-ci :

![Image](https://media.datacamp.com/legacy/v1714049144/image_b7d645a670.png)

Vous pouvez télécharger le PDF depuis cette page de mon GitHub.

L'étape suivante consiste à installer la bibliothèque pdf2image, qui nécessite un logiciel de traitement des PDF appelé Poppler. Voici les instructions spécifiques à chaque plate-forme :

Pour Windows, vous pouvez suivre les instructions de la documentation de PDF2Image.

Après l'installation, nous importons les modules appropriés :

In [16]:
import pathlib
from pathlib import Path

from pdf2image import convert_from_path

La fonction convert_from_path convertit un PDF donné en une série d'images. Voici une fonction qui enregistre chaque page d'un fichier PDF sous forme d'image dans un répertoire donné :

In [17]:
def pdf_to_image(pdf_path, output_folder: str = "."):
    """
    A function to convert PDF files to images
    """
    # Create the output folder if it doesn't exist
    if not Path(output_folder).exists():
        Path(output_folder).mkdir()

    pages = convert_from_path(pdf_path, output_folder=output_folder, fmt="png")

    return pages

Exécutons-le sur notre document :

In [18]:
import os

# Afficher le répertoire courant
print(os.getcwd())

c:\Users\pierr\VSC_Projects\Rpur


In [19]:
pdf_path = "scanned_document.pdf"

pdf_to_image(pdf_path, output_folder="documents")

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1662x2341>]

Le résultat est une liste contenant un seul objet image PngImageFile. Jetons un coup d'œil au répertoire documents:

In [23]:
ls documents

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 7069-5260

 R�pertoire de c:\Users\pierr\VSC_Projects\Rpur\documents

10/03/2025  22:13    <DIR>          .
10/03/2025  22:09    <DIR>          ..
10/03/2025  22:13         2�027�262 49e107d4-9cea-4b47-880f-e1ae44ce12cb-1.png
               1 fichier(s)        2�027�262 octets
               2 R�p(s)  79�204�216�832 octets libres


L'image est là, alors envoyons-la à la fonction image_to_text que nous avons créée au début et imprimons les quelques centaines de caractères du texte extrait :

In [ ]:
scanned_img_path = "../data/tutorial/documents/49e107d4-9cea-4b47-880f-e1ae44ce12cb-1.png"

print(image_to_text(scanned_img_path)[:377])

PFU Business report

New customer's development
and increasing the sale of product

My country economy at this season keeps escaping from Odoba of business though holds a crude oil
high so on unstable element that continues still, and recovering gradually and well.
In the IT industry, there is an influence such as competing intensification in narrowing investment field.

[Th


Si nous comparons le texte au fichier, tout fonctionne bien - le formatage et l'espacement sont préservés et le texte est exact. Comment partager le texte extrait ?

Le meilleur format pour partager un texte PDF extrait est un autre fichier PDF ! PyTesseract dispose d'une fonction image_to_pdf_or_hocr qui prend n'importe quelle image avec du texte et la convertit en un fichier PDF brut, consultable par le texte. Utilisons-le sur notre image numérisée :

In [28]:
raw_pdf = pytesseract.image_to_pdf_or_hocr(scanned_img_path)

with open("searchable_pdf.pdf", "w+b") as f:
    f.write(bytearray(raw_pdf))

Et voici à quoi ressemble le site searchable_pdf:

![Image](https://media.datacamp.com/legacy/v1714049439/image_bf9f396abc.gif)

Comme vous pouvez le constater, je peux surligner et copier du texte à partir du fichier. En outre, tous les éléments du PDF original sont préservés.

# <span style='background:blue'>Techniques de prétraitement d'images pour l'OCR dans OpenCV</span>

Il n'existe pas d'approche unique pour l'OCR. Les techniques que nous avons abordées aujourd'hui peuvent ne pas fonctionner avec d'autres types d'images. Je vous recommande d'expérimenter différentes techniques de prétraitement d'images et configurations de Tesseract afin de trouver les paramètres optimaux pour des images spécifiques.

Le facteur le plus important de l'OCR est la qualité de l'image. Les images correctement numérisées, entièrement verticales et très contrastées (en noir et blanc) sont celles qui fonctionnent le mieux avec les logiciels d'OCR. N'oubliez pas que ce n'est pas parce que vous pouvez lire le texte que votre ordinateur le peut.

Si vos images ne satisfont pas aux normes de qualité élevées de Tesseract et que le résultat est un charabia, vous pouvez effectuer quelques étapes de prétraitement.

## <span style='background:green'>1.  Conversion en niveaux de gris</span>


Commencez par convertir les images colorées en niveaux de gris. Cela permet d'améliorer la précision en supprimant les variations de couleur susceptibles de perturber le processus de reconnaissance. Dans OpenCV, cela ressemble à ceci :

In [29]:
def grayscale(image):
    """Converts an image to grayscale.

    Args:
        image: The input image in BGR format.

    Returns:
        The grayscale image.
    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

## <span style='background:green'>2.  Réduction du bruit</span>

Toutes les images, en particulier les documents numérisés, ne sont pas accompagnées d'un arrière-plan immaculé et uniforme. En outre, certaines images peuvent provenir de documents anciens dont les pages se sont détériorées en raison de l'âge. En voici un exemple :

![Image](https://media.datacamp.com/legacy/v1714049489/image_b38ea70997.png)

Appliquez des techniques telles que des filtres de débruitage (par exemple, le flou médian) pour réduire les artefacts de bruit dans l'image qui peuvent conduire à des erreurs d'interprétation lors de l'OCR. Dans OpenCV, vous pouvez utiliser la fonction medianBlur:

In [31]:
def denoise(image):
    """Reduces noise in the image using a median blur filter.

    Args:
        image: The input grayscale image.

    Returns:
        The denoised image.
    """
    return cv2.medianBlur(image, 5)  # Adjust kernel size as needed

## <span style='background:green'>3.  Affûtage</span>

Dans certains cas, l'accentuation de la netteté de l'image peut renforcer les contours et améliorer la reconnaissance des caractères, en particulier pour les images floues ou à faible résolution. L'accentuation peut être réalisée en appliquant un filtre Laplacien dans OpenCV :

In [32]:
def sharpen(image):
    """Sharpens the image using a Laplacian filter.

    Args:
        image: The input grayscale image.

    Returns:
        The sharpened image (be cautious with sharpening).
    """
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

## <span style='background:green'>4.  Binarisation</span>

Pour certaines images, la binarisation (conversion de l'image en noir et blanc) peut être bénéfique. Expérimentez différentes techniques de seuillage pour trouver la séparation optimale entre le premier plan (texte) et l'arrière-plan.

Cependant, la binarisation peut être sensible aux variations d'éclairage et n'est pas toujours nécessaire. Voici un exemple d'image binarisée :

![Image](https://media.datacamp.com/legacy/v1714049545/image_99bfaa0e01.png)

Pour effectuer une binarisation dans OpenCV, vous pouvez utiliser la fonction adaptiveThreshold:

In [33]:
def binarize(image):
    """Binarizes the image using adaptive thresholding.

    Args:
        image: The input grayscale image.

    Returns:
        The binary image.
    """
    thresh = cv2.adaptiveThreshold(
        image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2
    )

    return thresh

## <span style='background:green'>5.  Autres techniques</span>

Il existe de nombreuses autres techniques de prétraitement, telles que :

    Dilatation : rendre les polices petites et fines plus grasses pour améliorer la reconnaissance.
    Érosion : érosion du texte en gras pour améliorer la précision. Fréquent dans les documents historiques avec des polices de caractères épaisses.
    Détection d'arêtes Canny
    Correction de l'obliquité : correction de l'inclinaison (obliquité) des lignes de texte. Fréquent dans les documents mal numérisés

Vous pouvez en savoir plus sur les améliorations de la qualité de l'image en consultant cette page de la documentation de Tesseract.

# <span style='background:blue'>Conclusion</span>

Dans cet article, vous avez fait les premiers pas pour vous familiariser avec le problème dynamique qu'est l'OCR. Nous avons d'abord abordé la question de l'extraction de texte à partir d'images simples, puis nous sommes passés à des images plus difficiles avec un formatage complexe.

Nous avons également appris un processus de bout en bout pour extraire du texte à partir de PDF numérisés et comment enregistrer le texte extrait au format PDF pour qu'il puisse faire l'objet d'une recherche. Nous avons terminé l'article avec quelques conseils pour améliorer la qualité des images avec OpenCV avant de les envoyer à Tesseract.

Si vous souhaitez en savoir plus sur la résolution de problèmes liés à l'image, voici quelques ressources sur la vision par ordinateur :

    Cours sur le traitement d'images en Python
    Tutoriel sur les CNN en Python
    Traitement d'images avec le cursus Python - une collection de cours
    Tutoriel OpenCV en Python
